### m8.1stan

m8.1stan is the first model in the Statistical Rethinking book (pp. 249) using Stan.

Here we will use Turing's NUTS support, which is currently (2018) the originalNUTS by [Hoffman & Gelman]( http://www.stat.columbia.edu/~gelman/research/published/nuts.pdf) and not the one that's in Stan 2.18.2, i.e., Appendix A.5 in: https://arxiv.org/abs/1701.02434

The TuringModels pkg imports modules such as CSV and DataFrames

In [1]:
using TuringModels, Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


Read in the `rugged` data as a DataFrame

In [2]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');

Show size of the DataFrame (should be 234x51)

In [3]:
size(d)

(234, 51)

Apply log() to each element in rgdppc_2000 column and add it as a new column

In [4]:
d = hcat(d, map(log, d[Symbol("rgdppc_2000")]));

Rename our col x1 => log_gdp

In [5]:
rename!(d, :x1 => :log_gdp);

Now we need to drop every row where rgdppc_2000 == missing

When this (https://github.com/JuliaData/DataFrames.jl/pull/1546) hits DataFrame it'll be conceptually easier: i.e., completecases!(d, :rgdppc_2000)

In [6]:
notisnan(e) = !ismissing(e)
dd = d[map(notisnan, d[:rgdppc_2000]), :];

Updated DataFrame dd size (should equal 170 x 52)

In [7]:
size(dd)

(170, 52)

Define the Turing model

In [8]:
@model m8_1stan(y, x₁, x₂) = begin
    σ ~ Truncated(Cauchy(0, 2), 0, Inf)
    βR ~ Normal(0, 10)
    βA ~ Normal(0, 10)
    βAR ~ Normal(0, 10)
    α ~ Normal(0, 100)

    for i ∈ 1:length(y)
        y[i] ~ Normal(α + βR * x₁[i] + βA * x₂[i] + βAR * x₁[i] * x₂[i], σ)
    end
end;

Test to see that the model is sane. Use 2000 for now, as in the book.
Need to set the same stepsize and adapt_delta as in Stan...

Use Turing mcmc

In [9]:
posterior = sample(m8_1stan(dd[:log_gdp], dd[:rugged], dd[:cont_africa]),
Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.1
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.028298413339053828, std = [1.0, 1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 290.7372216720001;
  #lf / sample        = 0.0;
  #evals / sample     = 45.949;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0].


Describe the posterior samples

In [10]:
describe(posterior)

Iterations = 1:2000
Thinning interval = 1
Chains = 1
Samples per chain = 2000

Empirical Posterior Estimates:
              Mean           SD         Naive SE        MCSE         ESS    
       α    9.172685916  0.539672543 0.01206744491 0.05217665258  106.981386
  lf_num    0.000000000  0.000000000 0.00000000000 0.00000000000         NaN
      βA   -1.898779289  0.470268544 0.01051552433 0.05217559801   81.237582
      βR   -0.181017841  0.233840279 0.00522882759 0.02338156427  100.021074
       σ    0.968754734  0.232627447 0.00520170785 0.02032779040  130.960873
 elapsed    0.145368611  0.093850995 0.00209857205 0.00342077975  752.709743
 epsilon    0.028941493  0.013858204 0.00030987885 0.00021608833 2000.000000
eval_num   45.949000000 27.904349578 0.62396022524 0.87435326475 1018.520879
     βAR    0.373992527  0.207308860 0.00463556703 0.02010936414  106.276942
      lp -250.421713727 18.848907391 0.42147438228 1.83940573385  105.006767
  lf_eps    0.028941493  0.013858204 0.0003

Fix the inclusion of adaptation samples

In [11]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names)

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Union{Missing, Float64}[9.26481 0.0 … -247.272 0.0282984; 9.23935 0.0 … -246.169 0.0282984; … ; 9.15283 0.0 … -248.715 0.0282984; 9.10157 0.0 … -247.162 0.0282984]

Example of a Turing run simulation output

Here's the ulam() output from rethinking

In [12]:
m8_1s_cmdstan = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean         SD       Naive SE       MCSE      ESS
    a  9.22360053 0.139119116 0.0021996664 0.0034632816 1000
   bR -0.20196346 0.076106388 0.0012033477 0.0018370185 1000
   bA -1.94430980 0.227080488 0.0035904578 0.0057840746 1000
  bAR  0.39071684 0.131889143 0.0020853505 0.0032749642 1000
sigma  0.95036370 0.052161768 0.0008247500 0.0009204073 1000

Quantiles:
          2.5%       25.0%       50.0%      75.0%        97.5%
    a  8.95307475  9.12719750  9.2237750  9.31974000  9.490234250
   bR -0.35217930 -0.25334425 -0.2012855 -0.15124725 -0.054216855
   bA -2.39010825 -2.09894500 -1.9432550 -1.78643000 -1.513974250
  bAR  0.13496995  0.30095575  0.3916590  0.47887625  0.650244475
sigma  0.85376115  0.91363250  0.9484920  0.98405750  1.058573750
";

Describe the posterior samples

In [13]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                   SD                       Naive SE                       MCSE                ESS   
       α    9.220630044  0.146568537006979970893639 0.00463490410260735102160323 0.0086983648829756093912025  283.92672
  lf_num    0.000000000  0.000000000000000000000000 0.00000000000000000000000000 0.0000000000000000000000000        NaN
      βA   -1.927467763  0.234807850298384551690489 0.00742527619430742580330929 0.0172806868219145190468389  184.63041
      βR   -0.203183007  0.081130815900362585058048 0.00256558166672950140746456 0.0041346883572629311229263  385.02251
       σ    0.949919816  0.052648881216447632191979 0.00166490380903630942957538 0.0016653454543760844337730  999.46968
 elapsed    0.140624627  0.080469751548063558899138 0.00254467697639741263077040 0.0030369925297186453269638  702.06589
 epsilon    0.028298413  0.00000000000000001041354

end of 08/m8.1t.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*